In [1]:
import pandas as pd


In [3]:
lista = [{
    "user":"ivan",
    "password":"ivan123"
},
{
    "user":"javier",
    "password":"javier123"
}
]

df = pd.DataFrame(lista)

In [9]:
with open('auth.json', 'w') as f:
    f.write(df.to_json( orient='records', lines = True))


In [6]:
from datetime import datetime
user_nickname = 'carlos'
user_name = 'carlos jose'
user_last_name = 'maestre muza'
user_password = 'carlos123'
user_date_registration = datetime.now().strftime('%d/%m/%Y')
user_role = 'Administrator'

def new_user(user,password,name,last_name,role,date_login):
    new_user_data = {
        'user' : user,
        'password' : password,
        'name' : name,
        'last_name' : last_name,
        'role' : role,
        'date_login' : date_login,
    }

    df_temp = pd.DataFrame([new_user_data])
    df = pd.read_json('auth.json', lines=True)
    df = pd.concat([df,df_temp])

    with open('auth.json', 'w') as f:
        f.write(df.to_json( orient='records', lines = True))

In [10]:
from db import create_new_user
x = [{'user': 'ivan',
  'password': 'ivan123',
  'name': 'Ivan Daniel',
  'last_name': 'Maestre Muza',
  'role': 'administrator'},
 {'user': 'javier',
  'password': 'javier123',
  'name': 'Javier Javier',
  'last_name': 'Daza Olivella',
  'role': 'administrator'},
 {'user': 'eladio',
  'password': 'eladio123',
  'name': 'Eladio',
  'last_name': 'Carrión Morales',
  'role': 'viewer'}]

for i in x:
    user_name = i['user']
    password = i['password']
    name = i['name']
    last_name = i['last_name']
    role = i['role']
    create_new_user(user_name,password,name,last_name,role)

In [18]:


def authentication(user:str, password:str):
    response = False
    df = pd.read_json('auth.json', lines=True)
    df_dict = df.to_dict(orient='records')

    for dictionary in df_dict:

        if dictionary['user'] == user and dictionary['password'] == password:
            response = True

    return response
            


In [21]:
user = 'javier'
password = 'javier123'

authentication(user=user,password=password)

True

In [3]:
df = pd.read_json('auth.json', lines=True)
df_dict = df.to_dict(orient='records')
df_dict

[{'user': 'ivan',
  'password': 'ivan123',
  'name': 'Ivan Daniel',
  'last_name': 'Maestre Muza',
  'role': 'administrator'},
 {'user': 'javier',
  'password': 'javier123',
  'name': 'Javier Javier',
  'last_name': 'Daza Olivella',
  'role': 'administrator'},
 {'user': 'eladio',
  'password': 'eladio123',
  'name': 'Eladio',
  'last_name': 'Carrión Morales',
  'role': 'viewer'}]

In [1]:
from deta import Deta
from datetime import datetime
DETA_KEY = 'e0jeqtjtszl_L996atb2Q4i8HKS36bU6drgU6tt1HYSq' 

# Initialize with a project key

deta = Deta(DETA_KEY)


# This is how to create/connect a database
db = deta.Base("users")

def create_new_user(user_name,password,name,last_name,role):
    # create new user
    date_login = datetime.now().strftime('%d-%m-%Y')
    return db.put({"key":user_name, "password":password, "name":name.title().strip(), "last_name": last_name.title().strip(),"role": role.title().strip(),"date_login":date_login})


def update_user(user_name,password,name,last_name,role):
    return db.update({"password":password, "name":name.title().strip(), "last_name": last_name.title().strip(),"role": role.title().strip()}, key = user_name)

def fetch_all_users():
    """Returns a dict of all users"""
    res = db.fetch()
    return res.items


def get_user(user_name:str):
    return db.get(user_name)


def authentication(user_name_input:str, password_input):
    user_fetched = get_user(user_name_input)
    if user_fetched != None:
        password_fetched = user_fetched['password']
        if password_input == password_fetched:
            password = user_fetched['password']
            name = user_fetched['name']
            last_name = user_fetched['last_name']
            role = user_fetched['role']
            auth_flag = True
            return auth_flag,password,name,last_name,role
        else:
            password = ''
            name = ''
            last_name = ''
            role = ''
            auth_flag = False
            return auth_flag,password,name,last_name,role




In [36]:
key = "ivan"
password = "ivan123"
name = "Ivan daniel"
last_name = "maestre muza"
role = "administrator"


# if get_user(key) != None:
#     auth_flag,password,name,last_name,role = authentication(user_name_input= key, password_input = password)
# else:
#     print('The user dont exists')
create_new_user(key,password,name,last_name,role)

{'date_login': '18-02-2023',
 'key': 'ivan',
 'last_name': 'Maestre Muza',
 'name': 'Ivan Daniel',
 'password': 'ivan123',
 'role': 'Administrator'}

In [ ]:
def change_password(user_name_input,password_input,new_password_input:str):
    user_fetched = get_user(user_name_input)
    if user_fetched != None:
        try:
            db.update({"password":password_input},key=user_name_input)
            return True
        except Exception as e:
            print(e)
            return False

In [4]:
user_name_input = 'efra'
password_input = 'efra123'
new_password_input = 'efra124'


user_fetched = get_user(user_name_input)
if user_fetched != None:
    user_dict = db.get(user_name_input)
    if user_dict['password'] == password_input:
        try:
            db.update({"password":new_password_input},key=user_name_input)
            retorno =  True
        except Exception as e:
            print(e)
            retorno =  False
else:
    retorno = False

In [5]:
retorno

True

In [43]:
x = db.update({"password":'efra123'},key='efra')

In [3]:
db.get('efra')

{'date_login': '18-02-2023',
 'key': 'efra',
 'last_name': 'Aponte Restrepo',
 'name': 'Efra',
 'password': 'efra123',
 'role': 'Viewer'}

In [7]:
from db import change_password
change_password('efra','efra124','eee')

True

In [14]:
x = pd.DataFrame(fetch_all_users())
x = x[x['key'] == 'ivan']
y = x.to_dict(orient='records')
y

[{'date_login': '20-02-2023',
  'key': 'ivan',
  'last_name': 'Maestre Muza',
  'name': 'Ivan Daniel',
  'password': 'ivan123',
  'role': 'Administrator'}]

In [47]:
import random
from datetime import datetime, timedelta
aulas = ['aula 1','aula 2','aula 3','aula 4','aula 5']

dict_dummy = []
start_date = datetime.strptime('01/01/2023', '%d/%m/%Y')
for i in aulas:
    for dia in range(1,50):
        dict_ = {
            'aula':i,
            'medicion': random.randint(500,650),
            'datetime': start_date + timedelta(dia),
            'sensor' : f'{i}_sensor'
        }
        dict_dummy.append(dict_)

start_date = datetime.strptime('01/01/2023', '%d/%m/%Y')
end_date = datetime.strptime('20/02/2023', '%d/%m/%Y')
end_date - start_date

datetime.timedelta(days=50)

In [48]:
df = pd.DataFrame(dict_dummy)

In [68]:
import plotly.express as px
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.02)
aula = 'aula 1'
df = df[df['aula'] == f'{aula}']
fig = px.line(df, x="datetime", y="medicion",  color='aula')

fig.update_layout(title_text=f'Line Plot CO2 Levels \n Aula: {aula}', title_x=0.5,xaxis_title =  'Date',yaxis_title = 'Nivel de CO2',legend_title = 'Aula')
fig.add_hline(y=df['medicion'].mean())

fig.show()


398.0

In [45]:
df_temp = df[df['aula'] == 'aula 1']
df_temp.sort_values(by = ['datetime'], ascending = False, inplace =True)
df_temp

,aula,medicion,datetime,sensor
262,aula 1,514,2023-02-19,aula 1_sensor
261,aula 1,585,2023-02-18,aula 1_sensor
260,aula 1,576,2023-02-17,aula 1_sensor
259,aula 1,517,2023-02-16,aula 1_sensor
258,aula 1,561,2023-02-15,aula 1_sensor
...,...,...,...,...
4,aula 1,551,2022-06-06,aula 1_sensor
3,aula 1,596,2022-06-05,aula 1_sensor
2,aula 1,584,2022-06-04,aula 1_sensor
1,aula 1,538,2022-06-03,aula 1_sensor
